In [11]:
pip install geopandas

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 323 kB 318 kB/s eta 0:00:01
     |████████████████████████████████| 4.9 MB 167 kB/s eta 0:00:01
     |████████████████████████████████| 15.1 MB 10.0 MB/s eta 0:00:01
     |████████████████████████████████| 1.3 MB 6.6 MB/s eta 0:00:01
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [12]:
# loading packages
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd 
import matplotlib.colors as mcolors
from datetime import datetime
import re

In [13]:
mpl.style.use('fivethirtyeight')